In [ ]:
from pathlib import Path
from cayleypy_pancake.utils.paths import RunPaths

ROOT = Path(".").resolve()          # запускай ноут из корня репо
DATA_DIR = ROOT / "data"
OUT_DIR = ROOT / "runs"
OUT_DIR.mkdir(exist_ok=True)

paths = RunPaths(OUT_DIR)

TEST_PATH = DATA_DIR / "test.csv"
BASELINE_PATH = DATA_DIR / "baseline_submission.csv"
FINAL_PATH = DATA_DIR / "baseline_final.csv"

Данный набор функций реализует базовые утилиты для работы с перестановками и решениями задачи pancake sorting.

Функции parse_permutation, moves_to_str и moves_len отвечают за безопасное преобразование данных между форматами. Это позволяет единообразно работать с результатами классических алгоритмов, эвристик и ML-моделей в одном пайплайне.

Функция pancake_sort_moves реализует классический жадный алгоритм pancake sorting и используется как корректный базовый ориентир. Алгоритм работает за O(n^2) и гарантированно гарантированно строит допустимую последовательность префиксных разворотов для любой перестановки и служит эталоном для проверки корректности, оценки качества и сравнения более сложных методов.

In [ ]:
from cayleypy_pancake.baseline import parse_permutation, pancake_sort_moves

Этот блок подготавливает тестовый датасет и извлекает из него базовую структурную информацию — размер перестановки n. Мы явно восстанавливаем длину каждой перестановки из строкового представления, а затем считаем распределение по n, чтобы понимать, с какими размерами входов мы работаем. Это важно для анализа покрытия теста, стратификации результатов и последующего сравнения моделей по сложности входа.

In [ ]:
import pandas as pd

test_df = pd.read_csv(TEST_PATH)

test_df["n"] = test_df["permutation"].apply(lambda x: len(parse_permutation(x)))

n_counts = (
    test_df["n"]
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={"index": "n"})
)

n_counts


,n,count
0,5,5
1,12,200
2,15,200
3,16,200
4,20,200
5,25,200
6,30,200
7,35,200
8,40,200
9,45,200


# Heuristic without DL

Этот набор функций реализует эвристики качества состояния для pancake-задачи, быстрые примитивы применения ходов и проверки решения, улучшатель базового решения через beam search с отсечениями, и экспериментальный грид-поиск по гиперпараметрам эвристики и beam search.

Основная идея: берём базовое решение (по умолчанию классический pancake sort), используем его длину как верхнюю границу и пытаемся найти более короткий путь, исследуя пространство состояний ограниченным лучом (beam) и эвристическим приоритетом. Затем прогоняем это по множеству тест-кейсов и конфигураций, сохраняя метрики (gain, ok, время) для анализа.

breakpoints2 считает число «разрывов» в перестановке: соседние элементы, которые не идут подряд по значению (разность по модулю не равна 1). Дополнительно учитывается штраф, если первый элемент не равен 0, что усиливает давление на «правильный старт». Эта метрика используется как простая эвристика «насколько мы далеко от упорядоченного вида».

gap_h считает разрывы в последовательности, включая виртуальные границы -1 слева и n справа. Это стандартная идея для pancake-задачи: корректный порядок соответствует цепочке -1,0,1,...,n, и каждое нарушение смежности увеличивает оценку. В отличие от breakpoints2, здесь учитываются также граничные условия.

mix_h комбинирует две эвристики: gap_h как основную и breakpoints2 как добавочный штраф с весом alpha. Такая смесь позволяет регулировать «жёсткость» эвристики, балансируя чувствительность к локальным разрывам и предпочтение правильного префикса. Возвращается вещественное значение, чтобы удобно масштабировать вклад.

beam_improve_or_baseline_h пытается улучшить базовое решение ограниченным поиском (beam search) с эвристической оценкой f = g + w*h. Длина базового решения используется как текущая верхняя граница: все пути, которые уже не могут быть лучше, отсекаются. Если улучшение найдено, возвращается более короткий путь, иначе — исходный baseline.

Функция выполняет пакетный прогон экспериментов: на наборе кейсов перебирает все комбинации гиперпараметров (alpha, w, beam_width, depth). Для каждого запуска строится решение улучшателем, проверяется корректность и считаются метрики качества относительно baseline. Результаты собираются в таблицу для последующего анализа и выбора конфигурации.

In [ ]:
from cayleypy_pancake.experiments.grid import select_cases_per_n, run_grid


Этот блок формирует репрезентативный поднабор тестовых случаев по размерам перестановок и запускает полный грид-эксперимент по улучшению базовых решений. Для каждого выбранного размера n случайно отбираются несколько кейсов, после чего перебираются комбинации параметров эвристики и beam search. Результатом является таблица с метриками качества и времени, пригодная для анализа масштабируемости и подбора оптимальных конфигураций.

In [ ]:


n_list = [12, 15, 16, 20, 25, 30, 35, 40, 45, 50, 75, 100]
mini_df = select_cases_per_n(test_df, n_list, k=5, seed=42)

alphas = [0.0, 0.5, 1.0]
ws = [0.5, 1.0, 1.5]
beam_widths = [64, 128, 256]
depths = [64, 128, 192]

df = run_grid(
    mini_df,
    alphas=alphas,
    ws=ws,
    beam_widths=beam_widths,
    depths=depths,
    log=True,
    log_each=100,
    beam_log=True,
    beam_log_every_layer=5
)

df.head()
OUT_DIR.mkdir(exist_ok=True)
df.to_csv(OUT_DIR / "grid_results.csv", index=False)


[grid] cases=60 cfg_per_case=81 total_runs=4860

[case 1/60] id=71 n=12 base_len=19

[case 2/60] id=192 n=12 base_len=9

[case 3/60] id=106 n=12 base_len=16

[case 4/60] id=198 n=12 base_len=17

[case 5/60] id=116 n=12 base_len=18

[case 6/60] id=281 n=15 base_len=22

[case 7/60] id=299 n=15 base_len=20

[case 8/60] id=357 n=15 base_len=22

[case 9/60] id=368 n=15 base_len=20

[case 10/60] id=333 n=15 base_len=23

[case 11/60] id=527 n=16 base_len=20

[case 12/60] id=480 n=16 base_len=25

[case 13/60] id=459 n=16 base_len=26

[case 14/60] id=564 n=16 base_len=26

[case 15/60] id=600 n=16 base_len=28

[case 16/60] id=707 n=20 base_len=31

[case 17/60] id=792 n=20 base_len=29

[case 18/60] id=699 n=20 base_len=34

[case 19/60] id=660 n=20 base_len=29

[case 20/60] id=736 n=20 base_len=27

[case 21/60] id=870 n=25 base_len=38

[case 22/60] id=961 n=25 base_len=38

[case 23/60] id=948 n=25 base_len=46

[case 24/60] id=933 n=25 base_len=40

[case 25/60] id=903 n=25 base_len=38

[case 26/60]

,id,n,base_len,alpha,w,beam_width,depth,ok,steps,gain,time_sec
0,71,12,19,0.0,0.5,64,64,True,11,8,0.050677
1,71,12,19,0.0,0.5,64,128,True,11,8,0.136621
2,71,12,19,0.0,0.5,64,192,True,11,8,0.036501
3,71,12,19,0.0,0.5,128,64,True,11,8,0.074562
4,71,12,19,0.0,0.5,128,128,True,11,8,0.080702


Во всех конфигурациях beam search успешно находит корректные решения для всех кейсов (solved = 60), а средний выигрыш относительно baseline стабилен и лежит в диапазоне 28-29 шагов. Качество решения растёт с увеличением beam_width, но при этом время выполнения растёт существенно быстрее. Влияние параметров alpha и w на итоговое качество оказывается вторичным. Увеличение глубины поиска (depth) почти не даёт выигрыша, начиная с умеренных значений.

In [ ]:
summary = (
    df.groupby(["alpha","w","beam_width","depth"], as_index=False)
      .agg(
          solved=("ok","sum"),
          mean_gain=("gain","mean"),
          mean_steps=("steps","mean"),
          mean_time=("time_sec","mean"),
      )
      .sort_values(["solved","mean_gain","mean_time"], ascending=[False, False, True])
)

summary.head(20)


,alpha,w,beam_width,depth,solved,mean_gain,mean_steps,mean_time
7,0.0,0.5,256,128,60,28.583333,38.366667,12.573641
8,0.0,0.5,256,192,60,28.583333,38.366667,12.693728
16,0.0,1.0,256,128,60,28.566667,38.383333,12.376708
17,0.0,1.0,256,192,60,28.566667,38.383333,12.467067
34,0.5,0.5,256,128,60,28.550000,38.400000,16.833309
35,0.5,0.5,256,192,60,28.550000,38.400000,16.935335
61,1.0,0.5,256,128,60,28.533333,38.416667,16.524541
62,1.0,0.5,256,192,60,28.533333,38.416667,16.904485
31,0.5,0.5,128,128,60,28.183333,38.766667,7.376665
32,0.5,0.5,128,192,60,28.183333,38.766667,7.489081


Далее мы фиксируем alpha=0.0 (gap-эвристика) и w=0.5 и создаем сабмит из наилучших результатов трех beam search.

1. Лучшее качество: (alpha=0.0, w=0.5, bw=256, depth=128)

2. Максимальные beam_depth и beam_width : (alpha=0.0, w=0.5, bw=256, depth=128) (равны по mean_gain)

3. Быстрый и экономный: (alpha=0.0, w=0.5, bw=128, depth=128)

In [ ]:
from pathlib import Path
from cayleypy_pancake.eval import full_eval_top_cfgs

Выполним полный прогон выбранных конфигураций (top_cfgs) на множестве всех тестовых примеров с размерами из n_list и сохраним результаты в CSV.

In [ ]:
OUT_CSV = paths.out_dir / "full_eval_top_cfgs_seed42.csv"

n_list = [5, 12, 15, 16, 20, 25, 30, 35, 40, 45, 50, 75, 100]

top_cfgs = [
    {"alpha": 0.0, "w": 0.5, "beam_width": 128, "depth": 128},
    {"alpha": 0.0, "w": 0.5, "beam_width": 256, "depth": 128},
    {"alpha": 0.0, "w": 0.5, "beam_width": 256, "depth": 192},
]

df_full = full_eval_top_cfgs(
    test_df,
    n_list=n_list,
    top_cfgs=top_cfgs,
    out_csv_path=OUT_CSV,
    log=True,
    log_every=100,
)

df_full.head()


Эксперименты показывают, что увеличение ширины луча с 128 до 256 даёт лишь маргинальный прирост качества (≈0.27 шага в среднем), при этом увеличивая время выполнения более чем в два раза. Увеличение глубины поиска с 128 до 192 не оказывает измеримого влияния на качество.

Конфигурация beam_width=128, depth=128 выглядит как оптимальный компромисс между качеством и вычислительной стоимостью.

In [ ]:
summary_full = (
    df_full.groupby(["cfg_idx","alpha","w","beam_width","depth"], as_index=False)
           .agg(
               solved=("ok","sum"),
               mean_gain=("gain","mean"),
               mean_steps=("steps","mean"),
               mean_time=("time_sec","mean"),
           )
           .sort_values(["solved","mean_gain","mean_time"], ascending=[False, False, True])
)
summary_full


,cfg_idx,alpha,w,beam_width,depth,solved,mean_gain,mean_steps,mean_time
2,2,0.0,0.5,256,192,2405,27.859875,38.119335,11.731055
1,1,0.0,0.5,256,128,2405,27.859875,38.119335,11.741843
0,0,0.0,0.5,128,128,2405,27.585447,38.393763,4.977610


Функция сравнивает качество решений из submission_df с выбранным baseline-алгоритмом на полном test_df. Для каждого примера она считает длину baseline-решения и длину решения из сабмита, накапливая суммарные значения и статистику (сколько улучшили/ухудшили, максимальный выигрыш и т.п.).

In [ ]:
from cayleypy_pancake.eval import evaluate_submission_vs_baseline


Мы сравнили итоговый submission с baseline-алгоритмом (классический pancake sorting) на всём тестовом наборе из 2405 перестановок. Submission улучшает baseline в 2401 случаях (99.83%), не ухудшая результат ни в одном кейсе; ещё в 4 случаях длины совпадают. Суммарная длина решений снизилась с 158 680 до **91 594** ходов, что даёт общий выигрыш 67 086 ходов и средний выигрыш 27.94 шага на улучшенных примерах. Максимальный выигрыш на одном кейсе составил 92 шага (id=2342).

In [ ]:
sub_df = pd.read_csv("/content/drive/MyDrive/pancake_runs/submission_bs.csv")
stats = evaluate_submission_vs_baseline(
    test_df=test_df,
    submission_df=sub_df,
    baseline_moves_fn=pancake_sort_moves,
    log_every=50,
    save_detailed_path="sub_vs_base_details.csv",
)
stats

[   1/2405] base=  2 sub=  2 gain=  0  elapsed=    0.0s
[  50/2405] base= 19 sub=  8 gain= 11  elapsed=    0.0s
[ 100/2405] base= 20 sub= 12 gain=  8  elapsed=    0.0s
[ 150/2405] base= 16 sub= 11 gain=  5  elapsed=    0.0s
[ 200/2405] base= 15 sub= 11 gain=  4  elapsed=    0.0s
[ 250/2405] base= 19 sub= 15 gain=  4  elapsed=    0.0s
[ 300/2405] base= 20 sub= 14 gain=  6  elapsed=    0.0s
[ 350/2405] base= 20 sub= 15 gain=  5  elapsed=    0.0s
[ 400/2405] base= 21 sub= 14 gain=  7  elapsed=    0.0s
[ 450/2405] base= 21 sub= 13 gain=  8  elapsed=    0.0s
[ 500/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 550/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 600/2405] base= 21 sub= 12 gain=  9  elapsed=    0.0s
[ 650/2405] base= 32 sub= 20 gain= 12  elapsed=    0.0s
[ 700/2405] base= 34 sub= 19 gain= 15  elapsed=    0.0s
[ 750/2405] base= 32 sub= 19 gain= 13  elapsed=    0.0s
[ 800/2405] base= 33 sub= 19 gain= 14  elapsed=    0.0s
[ 850/2405] base= 42 sub= 24 gain= 18  elapsed= 

{'baseline_total': 158680,
 'submission_total': 91594,
 'total_gain': 67086,
 'improved_cases': 2401,
 'same_cases': 4,
 'worse_cases': 0,
 'avg_gain_when_improved': 27.9408579758434,
 'max_gain': 92,
 'max_gain_id': 2342,
 'time_sec': 0.2909247875213623,
 'sec_per_sample': 0.00012096664761803007,
 'mean_baseline_len': 65.97920997920998,
 'mean_submission_len': 38.08482328482329,
 'improved_frac': 0.9983367983367983}

Агрегация результатов по размерам перестановок показывает, что выигрыш относительно baseline растёт почти линейно с n. При n=12 средний выигрыш составляет около 5 шагов, тогда как при n=100 он превышает 80 шагов. При этом не наблюдается ни одного случая ухудшения решения. Более того, относительное сокращение длины решения увеличивается с ростом n, что указывает на хорошую масштабируемость предложенного апгрейда и его способность эффективно эксплуатировать структуру pancake-графа на больших размерах.

In [ ]:
from cayleypy_pancake.eval import evaluate_submission_vs_baseline
from cayleypy_pancake.baseline import pancake_sort_moves

sub_path = paths.out_dir / "submission_bs.csv"
sub_df = pd.read_csv(sub_path)

stats = evaluate_submission_vs_baseline(
    test_df=test_df,
    submission_df=sub_df,
    baseline_moves_fn=pancake_sort_moves,
    log_every=50,
    save_detailed_path=paths.out_dir / "sub_vs_base_details.csv",
)

stats


,n,mean_gain,improved,worse,mean_base,mean_sub
0,5,0.200,1,0,3.400,3.200
1,12,4.990,200,0,15.820,10.830
2,15,7.390,200,0,21.130,13.740
3,16,7.965,200,0,22.780,14.815
4,20,11.550,200,0,30.460,18.910
5,25,15.740,200,0,39.615,23.875
6,30,20.625,200,0,49.550,28.925
7,35,24.745,200,0,58.750,34.005
8,40,29.235,200,0,68.470,39.235
9,45,32.975,200,0,77.450,44.475


Классический pancake sorting быстро становится неоптимальным при росте n, тогда как ограниченный эвристический поиск находит всё более короткие траектории, причём выигрыш растёт пропорционально размеру задачи.

# ML-heuristic with DL

In [ ]:
!pip install git+https://github.com/cayleypy/cayleypy.git@e1518b6 --no-deps

  Cloning https://github.com/cayleypy/cayleypy.git (to revision e1518b6) to /tmp/pip-req-build-rxi6fpu1
  Running command git clone --filter=blob:none --quiet https://github.com/cayleypy/cayleypy.git /tmp/pip-req-build-rxi6fpu1
  Running command git checkout -q e1518b6
  Resolved https://github.com/cayleypy/cayleypy.git to commit e1518b6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cayleypy: filename=cayleypy-0.1.0-py3-none-any.whl size=592103 sha256=ba6530e7983b2908468466a2ec1fce46ee6cd69e33c8f571d908835d23ed90dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-cf_l8bti/wheels/bb/02/6c/551e0b1e957367d2581e24443666bf3222295029a6821a4726
Successfully built cayleypy


Далее рассмотрим семейство регрессионных моделей.
Реализованы три варианта представления входа:
1. One-hot MLP (простая базовая)
2. One-hot MLP с residual-стеком
3. Embedding-версия

Embedding-регрессор заменяет one-hot представление на обучаемые эмбеддинги токенов (значений перестановки), что обычно существенно экономит память и ускоряет вычисления при больших n. Дополнительно может добавляться позиционный эмбеддинг, позволяющий модели различать одинаковые значения в разных позициях и лучше реагировать на структуру перестановки. После эмбеддингов последовательность разворачивается в вектор и обрабатывается MLP (как в Pilgrim), включая опциональные residual-блоки.

 Функция get_model выбирает архитектуру по cfg["model_type"], позволяя переключать модели без изменения остального пайплайна.


Зададим вспомогательные утилиты для воспроизводимых экспериментов и удобной организации артефактов. Здесь фиксируется базовая конфигурация обучения (BASE_CFG), предоставляется способ порождать конфиги с переопределениями (make_cfg) и строится сетка архитектурных вариантов для перебора (build_model_grid). Дополнительно формируется читаемое имя эксперимента (exp_name_from), чтобы результаты можно было систематизировать по параметрам модели.

In [ ]:
from cayleypy_pancake.config import build_model_grid

Реализуем обучение регрессионной модели на данных, сгенерированных из pancake-графа через случайные обходы (random walks). На каждой эпохе заново генерируется датасет состояний X и целевых значений y, применяется трансформация таргета (например, log1p), после чего данные перемешиваются и делятся на train/val.

Обучение идёт на GPU с AdamW, SmoothL1Loss, клиппингом градиента и CosineAnnealingLR. Sanity-check оценивает корреляцию между предсказаниями модели и лог-трансформированным таргетом, сгенерированным в BFS-режиме. Это быстрый индикатор того, что модель “видит сигнал” и не выдаёт константу или шум. Использование @torch.no_grad() делает функцию дешёвой и безопасной для запуска между эпохами/экспериментами.

Идея: использовать обученную модель как эвристику для поиска. Модель выдаёт оценку “насколько состояние далеко от цели”, и эта оценка используется внутри beam search для ранжирования кандидатов.

В отличие от чисто ручной эвристики, здесь ML-оценка считается батчами и комбинируется с лёгкой структурной метрикой gap_h для стабилизации.

Функция beam_improve_with_ml пытается улучшить baseline-решение через beam search, используя ML-эвристику h_fn как основное ранжирование кандидатов. Длина baseline (best_len) задаёт верхнюю границу: все пути, которые уже не могут стать лучше baseline, отсекаются. Возвращается улучшенный путь, если найден, иначе — baseline, что гарантирует “не хуже baseline” в нормальном режиме работы

Дополнительно предусмотрены утилиты для формирования подвыборок тестовых примеров фиксированного размера n и для оценки выигрыша ML-поиска относительно baseline.

In [ ]:

from cayleypy_pancake.baseline import pancake_sort_moves


Реализуем функцию для проведения экспериментов с прогоном по нескольким размерам n и сетке архитектур model_grid. Для каждого n строится pancake-граф, формируется фиксированный набор тест-кейсов, затем обучается модель на сгенерированных random-walk данных и используется как эвристика внутри beam search.


Мы оцениваем модель по конечному выигрышу в длине решения при использовании её как эвристики в beam search.

In [ ]:
from cayleypy_pancake.ml.pipeline import run_sweep_n_list

Зададим список n, на которых будет проводиться ML-оценка. Эти значения представляют малый, средний и более сложный режимы, позволяя увидеть, как масштабирование влияет на качество эвристики.

Для каждого n сравниваются разные архитектуры (embedding-модели и one-hot MLP) по их способности улучшать baseline-решение в beam search. Варьируем размер эмбеддингов, наличие позициционных эмбеддингов и количество residual блоков.

In [ ]:
from pathlib import Path

In [ ]:
n_list = [20, 30, 50]

model_grid = build_model_grid(
    emb_dims=(32, 64),
    pos_opts=(True, False),
    nrds=(0, 2, 6),
    include_mlpres1=True,
)

df = run_sweep_n_list(
    test_df=test_df,
    n_list=n_list,
    rows_k=20,
    rows_seed=42,
    model_grid=model_grid,
    w_gap=0.15,
    gap_mode="log1p",
    results_csv=paths.out_dir / "ml_sweep" / "result_n20_30_50.csv",
    out_json_dir=paths.out_dir / "ml_sweep" / "json",
    beam_width=256,
    depth=192,
    w=0.5,
)
df.head()


Представление перестановки через embedding элементов (и опционально позиций) существенно лучше one-hot кодирования для этой задачи.

Positional embedding не обязателен на малых n. Увеличение nrd с 2 до 6 не даёт выигрыша, но увеличивает время обучения. emb_dim=64 слегка лучше 32, но разница минимальна.

при росте n модель начинает выигрывать от информации о позиции элемента, а не только от его значения.

На больших n требуется большая глубина модели (nrd=6), emb_dim=32 оказывается лучше 64 (интересный эффект регуляризации). Positional embedding уже не критичен, а иногда даже ухудшает результат. Цена выигрыша - очень серьёзное время поиска.

In [ ]:
best_per_n = (
    df.sort_values(
        ["target_n", "total_gain", "sanity_corr"],
        ascending=[True, False, False]
    )
    .groupby("target_n", as_index=False)
    .head(5)
)

cols = [
    "target_n", "exp_name", "model_type",
    "emb_dim", "use_pos_emb", "nrd",
    "total_gain", "mean_ml_len",
    "sanity_corr", "train_time_sec", "time_sec_eval"
]

best_per_n[cols]


,target_n,exp_name,model_type,emb_dim,use_pos_emb,nrd,total_gain,mean_ml_len,sanity_corr,train_time_sec,time_sec_eval
11,20,n20_EmbMLP_ed64_pos0_nrd6_wg0.15_log1p,EmbMLP,64.0,False,6,229,18.70,0.924308,53.385489,18.869771
10,20,n20_EmbMLP_ed64_pos0_nrd2_wg0.15_log1p,EmbMLP,64.0,False,2,229,18.70,0.923873,34.667781,17.854649
7,20,n20_EmbMLP_ed64_pos1_nrd2_wg0.15_log1p,EmbMLP,64.0,True,2,228,18.75,0.927537,35.715142,18.047640
4,20,n20_EmbMLP_ed32_pos0_nrd2_wg0.15_log1p,EmbMLP,32.0,False,2,228,18.75,0.922724,34.754986,17.334467
8,20,n20_EmbMLP_ed64_pos1_nrd6_wg0.15_log1p,EmbMLP,64.0,True,6,227,18.80,0.924568,54.465402,18.901072
22,30,n30_EmbMLP_ed64_pos1_nrd2_wg0.15_log1p,EmbMLP,64.0,True,2,386,29.90,0.953272,64.946194,56.314097
25,30,n30_EmbMLP_ed64_pos0_nrd2_wg0.15_log1p,EmbMLP,64.0,False,2,374,30.50,0.950571,63.166497,58.021820
23,30,n30_EmbMLP_ed64_pos1_nrd6_wg0.15_log1p,EmbMLP,64.0,True,6,369,30.75,0.948167,95.269771,61.951936
16,30,n30_EmbMLP_ed32_pos1_nrd2_wg0.15_log1p,EmbMLP,32.0,True,2,368,30.80,0.948684,63.761750,57.413650
19,30,n30_EmbMLP_ed32_pos0_nrd2_wg0.15_log1p,EmbMLP,32.0,False,2,367,30.85,0.949092,61.264268,57.142353


One-hot MLP существенно уступает embedding-подходу для задачи pancake-графа, даже при увеличении глубины.
Однако глубина (residual stack)также критична для качества ML-эвристики.

emb_dim=32 стабильно лучше, чем 64.
Более компактное представление элемента оказывается эффективнее для эвристического поиска.

positional embedding не является определяющим фактором

In [ ]:
agg = (
    df.groupby(
        ["model_type", "emb_dim", "use_pos_emb", "nrd"],
        dropna=False,
        as_index=False
    )
    .agg(
        total_gain_sum=("total_gain", "sum"),
        mean_ml_len_mean=("mean_ml_len", "mean"),
        sanity_corr_mean=("sanity_corr", "mean"),
    )
    .sort_values(
        ["total_gain_sum", "sanity_corr_mean"],
        ascending=[False, False],
    )
)

agg.head(20)


,model_type,emb_dim,use_pos_emb,nrd,total_gain_sum,mean_ml_len_mean,sanity_corr_mean
2,EmbMLP,32.0,False,6,997,38.816667,0.939317
5,EmbMLP,32.0,True,6,978,39.133333,0.938082
11,EmbMLP,64.0,True,6,951,39.583333,0.944556
8,EmbMLP,64.0,False,6,948,39.633333,0.944285
4,EmbMLP,32.0,True,2,918,40.133333,0.941958
1,EmbMLP,32.0,False,2,900,40.433333,0.942980
7,EmbMLP,64.0,False,2,886,40.666667,0.943780
10,EmbMLP,64.0,True,2,861,41.083333,0.945967
14,MLPRes1,NaN,NaN,6,686,44.000000,0.920993
13,MLPRes1,NaN,NaN,2,596,45.500000,0.926183


Лучшей архитектурой в среднем по всем протестированным размерам задач является:

**EmbMLP с emb_dim = 32 и nrd = 6**

При этом использование positional embedding остаётся опциональным.

Эта модель демонстрирует оптимальный баланс между выразительностью, обобщающей способностью и практической эффективностью в задаче эвристического поиска в pancake-графе.

Приведенная ниже функция формирует строковый идентификатор эксперимента на основе конфигурации модели, размера задачи n и параметров эвристического поиска. Для embedding-моделей в имя включаются размер эмбеддинга, наличие positional embedding и глубина residual-стека, а для остальных моделей — только тип и глубина.

Дополнительно кодируются параметры генерации данных (rw_width, mix_bfs_frac), обучения (learning rate, weight decay, число эпох) и параметры эвристики (w_gap, gap_mode), что гарантирует уникальность и воспроизводимость экспериментов.

Для более тонкой настройки мы фиксируем лучшую по прошлым экспериментам архитектуру (EmbMLP с emb_dim=32 и nrd=6) и перебираем только параметры обучения и генерации данных, чтобы улучшить качество ML-эвристики. Перебор запускается на одном выбранном размере n (здесь n=30) как компромиссе между информативностью и вычислительной стоимостью.

В этом режиме различия между конфигурациями обучения наиболее отчётливо проявляются в итоговом выигрыше поиска, при этом вычислительная стоимость остаётся приемлемой.

Рассматриваются следующие параметры:
rw_width (регулирует объём сгенерированных данных на эпоху), mix_frac_grid (доля BFS-примеров в смешанном датасете), а lr и weight_decay (поведение оптимизатора и регуляризацию).

In [ ]:
from cayleypy_pancake.config import build_leader_train_grid
from cayleypy_pancake.ml.pipeline import run_sweep_n_list

LEADER_ARCH = dict(
    model_type="EmbMLP",
    emb_dim=32,
    use_pos_emb=False,
    hd1=512, hd2=256, nrd=6,
    dropout_rate=0.1,
)

TRAIN_BASE = dict(
    rw_mode="mix",
    nbt_history_depth=1,
    k=4,
    rw_length_add=30,
    y_transform="log1p",
    loss_beta=1.0,
    grad_clip=1.0,
    batch_size=1024,
    val_ratio=0.15,
    stratify_clip=60,
    stratify_bin_size=1.0,
    sanity_width=1200,
    h_batch_size=8192,
    eval_log_every=10,
    seed=123,
)

model_grid = build_leader_train_grid(
    leader_arch=LEADER_ARCH,
    train_base=TRAIN_BASE,
    rw_width_grid=[2000, 3000, 5000],
    mix_frac_grid=[0.15, 0.30, 0.50],
    lr_grid=[1e-4, 2e-4],
    weight_decay_grid=[5e-3, 1e-2],
    epochs_grid=[40],
)
print("Total training configs:", len(model_grid))

df_train = run_sweep_n_list(
    test_df=test_df,
    n_list=[30],
    rows_k=20,
    rows_seed=42,
    model_grid=model_grid,
    w_gap=0.15,
    gap_mode="log1p",
    results_csv=paths.out_dir / "leader_tune" / "results_train.csv",
    out_json_dir=paths.out_dir / "leader_tune" / "json",
    device=None,
    beam_width=256,
    depth=192,
    w=0.5,
)


Total training configs: 36
[resume] 3 experiments already done

########## TARGET_N = 30 ##########

===== [1/36] n30_EmbMLP_ed32_pos0_nrd6_rw2000_mix0.15_lr0.0001_wd0.005_ep40_wg0.15_log1p =====

[2026-02-04 16:52:31] EXP=n30_EmbMLP_ed32_pos0_nrd6_rw2000_mix0.15_lr0.0001_wd0.005_ep40_wg0.15_log1p | model=EmbMLP n=30 | epochs=40 rw_width=2000 rw_mode=mix
Epoch 000/40 | lr=9.98e-05 | train=0.40582 | val=0.16431
Epoch 001/40 | lr=9.94e-05 | train=0.17577 | val=0.14244
Epoch 002/40 | lr=9.86e-05 | train=0.13400 | val=0.10224
Epoch 003/40 | lr=9.76e-05 | train=0.10600 | val=0.07638
Epoch 004/40 | lr=9.62e-05 | train=0.08999 | val=0.06478
Epoch 005/40 | lr=9.46e-05 | train=0.07892 | val=0.05690
Epoch 006/40 | lr=9.26e-05 | train=0.06621 | val=0.05368
Epoch 007/40 | lr=9.05e-05 | train=0.06195 | val=0.04514
Epoch 008/40 | lr=8.80e-05 | train=0.06069 | val=0.04640
Epoch 009/40 | lr=8.54e-05 | train=0.05650 | val=0.04236
Epoch 010/40 | lr=8.25e-05 | train=0.05341 | val=0.04456
Epoch 011/40 | l

,exp_name,timestamp,target_n,rows_k,model_type,emb_dim,use_pos_emb,hd1,hd2,nrd,...,same_cases,worse_cases,avg_gain_when_improved,max_gain,max_gain_id,time_sec_eval,sec_per_sample_eval,mean_baseline_len,mean_ml_len,improved_frac
34,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.30_lr0.0...,2026-02-04 18:48:18,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.6,28,1157,69.792297,3.489615,49.2,29.6,1.0
29,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,2026-02-04 18:23:51,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.6,28,1157,69.195672,3.459784,49.2,29.6,1.0
37,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,2026-02-04 19:03:07,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.5,28,1157,69.276130,3.463806,49.2,29.7,1.0
36,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,2026-02-04 18:58:12,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.4,27,1157,70.756788,3.537839,49.2,29.8,1.0
30,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,2026-02-04 18:28:40,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.4,27,1157,68.937713,3.446886,49.2,29.8,1.0


При фиксированной архитектуре EmbMLP (emb_dim=32, nrd=6) наибольшее влияние на качество эвристики оказывают параметры генерации обучающих данных.
Увеличение ширины случайных обходов улучшает итоговый выигрыш.

Оптимальным оказался режим смешивания BFS и NBT с долей BFS около 30%, а также более агрессивные параметры оптимизации (lr=2e-4, weight decay=1e-2).


In [ ]:
best_gain = (
    df_train
    .sort_values(["total_gain", "sanity_corr"], ascending=[False, False])
    .head(39)
)

best_gain[
    ["total_gain","exp_name","rw_width","lr",
     "weight_decay","mix_bfs_frac",
     "sanity_corr","train_time_sec","time_sec_eval"]
]


,total_gain,exp_name,rw_width,lr,weight_decay,mix_bfs_frac,sanity_corr,train_time_sec,time_sec_eval
34,392,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.30_lr0.0...,5000,0.0002,0.010,0.30,0.955797,225.533947,69.792297
29,392,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,5000,0.0002,0.005,0.15,0.949510,223.916882,69.195672
37,390,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0002,0.005,0.50,0.958488,225.368810,69.276130
36,388,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0001,0.010,0.50,0.958075,224.477674,70.756788
30,388,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,5000,0.0002,0.010,0.15,0.950487,219.868918,68.937713
38,387,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0002,0.010,0.50,0.958318,222.863929,69.362046
22,386,n30_EmbMLP_ed32_pos0_nrd6_rw3000_mix0.30_lr0.0...,3000,0.0002,0.010,0.30,0.955731,143.221617,70.109416
26,386,n30_EmbMLP_ed32_pos0_nrd6_rw3000_mix0.50_lr0.0...,3000,0.0002,0.010,0.50,0.955653,142.869051,69.949446
17,386,n30_EmbMLP_ed32_pos0_nrd6_rw3000_mix0.15_lr0.0...,3000,0.0002,0.005,0.15,0.954151,140.495360,69.854921
35,385,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0001,0.005,0.50,0.958345,226.512267,70.597412


Рассчитаем и сохраним базовое решение(baseline) для всех тестовых перестановок, а также реализуем механизм продолжения работы через файл прогресса. Если прогресс уже существует, код подхватывает ранее посчитанные ответы и не пересчитывает их заново. Если прогресса нет, строится baseline_map для всех id в test_df и сохраняется на диск, чтобы дальше отталкиваться от него при улучшениях.

In [ ]:
from cayleypy_pancake.utils.progress import load_progress_map
from cayleypy_pancake.eval import build_and_save_baseline_submission

progress_map = load_progress_map(paths.progress_path)
print("Resume progress:", len(progress_map)) if progress_map else print("No progress, start fresh.")

if not (paths.baseline_path).exists():
    _ = build_and_save_baseline_submission(
        test_df,
        out_path=paths.baseline_path,
        baseline_moves_fn=pancake_sort_moves,
        log_every=2000,
    )
    print("Saved baseline:", paths.baseline_path)


No progress, start fresh.
  baseline [2000/2405] dt=0.1s
  baseline [2405/2405] dt=0.2s
Saved baseline: /content/drive/MyDrive/pancake_runs/baseline_submission.csv


Зафиксируем "финальную" конфигурацию пайплайна: параметры beam search и параметры обучения/использования ML-эвристики, выбранные по результатам тюнинга.

In [ ]:
from cayleypy_pancake.presets import LEADER_CFG, DEFAULT_BEAM

BEAM_WIDTH = DEFAULT_BEAM["beam_width"]
DEPTH = DEFAULT_BEAM["depth"]
W = DEFAULT_BEAM["w"]
W_GAP = DEFAULT_BEAM["w_gap"]
GAP_MODE = DEFAULT_BEAM["gap_mode"]
H_BATCH = DEFAULT_BEAM["h_batch_size"]


Реализуем финальный прогон: для каждого размера n из списка мы обучаем отдельную ML-эвристику на соответствующем pancake-графе, затем применяем её к всем тестовым кейсам этого размера и улучшаем baseline-решение через beam search.

In [ ]:
from cayleypy_pancake.presets import DEFAULT_BEAM
from cayleypy_pancake.ml.submit import build_submission_with_ml
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

final_df = build_submission_with_ml(
    test_df=test_df,
    n_list=n_list,
    leader_cfg=LEADER_CFG,
    device=device,
    beam_width=DEFAULT_BEAM["beam_width"],
    depth=DEFAULT_BEAM["depth"],
    w=DEFAULT_BEAM["w"],
    w_gap=DEFAULT_BEAM["w_gap"],
    gap_mode=DEFAULT_BEAM["gap_mode"],
    h_batch_size=DEFAULT_BEAM["h_batch_size"],
    baseline_path=paths.baseline_path,
    progress_path=paths.progress_path,
    final_path=paths.final_path,
    flush_every=200,
)
final_df.head()


Для задач малого и среднего размера (n < 75) beam search с обученной ML-эвристикой достигает того же качества решений, что и beam search с простой gap-эвристикой. На больших n поиск не увенчался успехом.

In [ ]:
stats = evaluate_submission_vs_baseline(
    test_df=test_df,
    submission_df=final_df,
    baseline_moves_fn=pancake_sort_moves,
    log_every=50,
    save_detailed_path="sub_vs_base_details.csv",
)
stats

[   1/2405] base=  2 sub=  2 gain=  0  elapsed=    0.0s
[  50/2405] base= 19 sub=  8 gain= 11  elapsed=    0.0s
[ 100/2405] base= 20 sub= 12 gain=  8  elapsed=    0.0s
[ 150/2405] base= 16 sub= 11 gain=  5  elapsed=    0.0s
[ 200/2405] base= 15 sub= 11 gain=  4  elapsed=    0.0s
[ 250/2405] base= 19 sub= 15 gain=  4  elapsed=    0.0s
[ 300/2405] base= 20 sub= 14 gain=  6  elapsed=    0.0s
[ 350/2405] base= 20 sub= 15 gain=  5  elapsed=    0.0s
[ 400/2405] base= 21 sub= 14 gain=  7  elapsed=    0.0s
[ 450/2405] base= 21 sub= 13 gain=  8  elapsed=    0.0s
[ 500/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 550/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 600/2405] base= 21 sub= 12 gain=  9  elapsed=    0.0s
[ 650/2405] base= 32 sub= 20 gain= 12  elapsed=    0.0s
[ 700/2405] base= 34 sub= 19 gain= 15  elapsed=    0.0s
[ 750/2405] base= 32 sub= 19 gain= 13  elapsed=    0.0s
[ 800/2405] base= 33 sub= 19 gain= 14  elapsed=    0.0s
[ 850/2405] base= 42 sub= 24 gain= 18  elapsed= 

{'baseline_total': 158680,
 'submission_total': 89252,
 'total_gain': 69428,
 'improved_cases': 2213,
 'same_cases': 192,
 'worse_cases': 0,
 'avg_gain_when_improved': 31.372797107998192,
 'max_gain': 195,
 'max_gain_id': 2210,
 'time_sec': 0.3336763381958008,
 'sec_per_sample': 0.00013874276016457413,
 'mean_baseline_len': 65.97920997920998,
 'mean_submission_len': 37.11101871101871,
 'improved_frac': 0.9201663201663202}

Создадим функцию, объединяющую несколько сабмитов, как полных, так и частичных (где решения есть только для части id), в один финальный файл.

Сначала выбираем base-файл, затем поверх него применяем partial-файлы, заменяя решение только если оно короче. В конце сохраняем merged submission и печатаем сводку, включая итоговый score (сумму длин) и распределение "победителей" по источникам.

In [ ]:
from cayleypy_pancake.submit import merge_submissions_with_partials

out_df = merge_submissions_with_partials(
    base_paths=[BASELINE_PATH, FINAL_PATH],
    partial_paths=["submission_n30.csv", "submission_n50.csv"],
    out_path="submission_final.csv",
    save_source_column=True,
    tie_break="keep_base",
)

out_df.head(10)
out_df["source"].value_counts().head(20)


В качестве главного сабмита используется один baseline-файл, и он служит опорной точкой, от которой мы будем отталкиваться при улучшениях.

В качестве дополнительных используем результат beam search с gap-эвристикой и результат ML-эвристики, сохранённый по ходу прогресса. Эти файлы не обязаны покрывать весь датасет и применяются поверх baseline только там, где они действительно улучшают решение.

In [ ]:
from pathlib import Path
from cayleypy_pancake.submit import merge_submissions_with_partials

RUNS_DIR = Path("runs")

BASE_PATHS = [
    RUNS_DIR / "submission_baseline.csv",
]

PARTIAL_PATHS = [
    RUNS_DIR / "submission_gap.csv",
    RUNS_DIR / "submission_progress_ml.csv",
]

merged_df = merge_submissions_with_partials(
    base_paths=BASE_PATHS,
    partial_paths=PARTIAL_PATHS,
    out_path=RUNS_DIR / "submission_final.csv",
    save_source_column=True,
    tie_break="keep_base",
)

merged_df.head()
merged_df["source"].value_counts().head(20)



=== MERGE SUMMARY ===
Output: submission_final.csv
Rows: 2405
Total moves (score): 91584

Final winners by source tag (top):


,count
source,
gap,2391
progress_ml,10
baseline,4



Saved: submission_final.csv


,id,solution,source
0,0,R4.R2,baseline
1,1,R5.R4.R2.R4,gap
2,2,R4.R2.R3.R2,baseline
3,3,R2.R3,baseline
4,4,R3.R5.R3.R4,baseline
5,5,R3.R2.R12.R5.R11.R8.R5.R9.R5.R10.R3.R7,gap
6,6,R10.R11.R4.R7.R6.R8.R10.R3.R12.R8,gap
7,7,R11.R6.R12.R4.R5.R3.R10.R3.R11.R5.R8,gap
8,8,R4.R7.R10.R6.R8.R6.R12.R9.R8.R4,gap
9,9,R10.R3.R6.R9.R8.R11.R4.R12.R7.R2.R5,gap


Подавляющее большинство финальных решений (2391 из 2405) пришло из beam search с gap-эвристикой. Это подтверждает, что для основного диапазона задач именно gap-эвристика даёт наиболее стабильные и воспроизводимые улучшения.

Фактически, именно она формирует "скелет" итогового сабмита.

Хотя вклад ML-эвристики количественно небольшой (10 кейсов), он ненулевой: все замены - строгие улучшения, ML действительно находит решения, которые gap-эвристика пропускает.

Финальный сабмит получен путём безопасного объединения baseline-решений с результатами двух улучшенных методов: beam search с gap-эвристикой и beam search с ML-эвристикой. Замены выполнялись только при строгом улучшении, что полностью исключает деградации.

Наилучший результат достигается гибридной стратегией:
beam search с ручной gap-эвристикой как основным методом и ML-эвристикой как дополнительным источником точечных улучшений.


Итоговый score: **91584**

# Про ML-модели

Во всех экспериментах модели класса EmbMLP значительно опережают one-hot MLP (MLPRes1) по суммарному выигрышу в длине решения. Даже при увеличении глубины и сложности one-hot архитектуры остаются заметно слабее embedding-подхода. Это указывает на то, что явное представление элементов перестановки в виде эмбеддингов является более подходящим способом кодирования состояний pancake-графа.

Число residual-блоков (nrd) оказывает наибольшее влияние на итоговый результат. Модели с nrd=6 стабильно демонстрируют наилучшие показатели суммарного выигрыша, тогда как уменьшение глубины приводит к заметной деградации качества. Это говорит о том, что для построения эффективной эвристики требуется достаточно выразительная нелинейная модель, способная захватывать сложные зависимости между элементами перестановки.

Внезапно, модели с размерностью эмбеддинга 32 в среднем превосходят модели с emb_dim=64. Более компактное представление элементов перестановки оказывается менее склонным к переобучению на данных случайных обходов и лучше переносится на задачу эвристического поиска. Этот результат подчёркивает, что увеличение выразительности модели не всегда приводит к улучшению её практической эффективности.

Использование positional embedding даёт лишь незначительный эффект и не является критичным для достижения высоких результатов. В некоторых режимах (средние значения n) оно может слегка улучшать качество, однако в среднем влияние позиционных эмбеддингов оказывается слабым по сравнению с вкладом глубины модели и способа кодирования состояний.

Хотя высокая корреляция предсказаний модели с BFS-дистанциями необходима для успешной эвристики, она не гарантирует максимального выигрыша в beam search. Некоторые модели с более высокой sanity_corr показывают меньший суммарный выигрыш, чем более «простые» архитектуры. Это подтверждает важность оценки моделей по конечному эффекту в поиске, а не только по регрессионным метрикам.